Feature selection is the process of selecting a subset of relevant features from the original set of features to improve the performance of a supervised learning classification model. Here are some commonly used methods for feature selection:

1. Univariate Feature Selection:
   - This method examines the relationship between each feature and the target variable independently.
   - Statistical tests such as chi-square test, ANOVA, or correlation coefficients are used to rank the features based on their relevance to the target.
   - SelectKBest and SelectPercentile are popular univariate feature selection techniques.

2. Recursive Feature Elimination (RFE):
   - RFE is an iterative method that starts with all features and eliminates the least important features in each iteration.
   - It uses a model (e.g., logistic regression, support vector machines) to determine the importance of features and eliminates the least important ones.
   - RFE continues the elimination process until a desired number of features is reached.
   - The sklearn library in Python provides the RFE implementation.

3. Feature Importance:
   - Some models provide feature importance scores that indicate the relevance or contribution of each feature to the prediction.
   - Random Forests and Gradient Boosting models, such as XGBoost and LightGBM, provide feature importance scores.
   - Features with higher importance scores are considered more relevant and can be selected for the final model.

4. L1 Regularization (Lasso):
   - L1 regularization adds a penalty term based on the absolute values of the feature coefficients in the model.
   - It encourages sparsity by shrinking less important features' coefficients to zero, effectively performing feature selection.
   - Models such as Logistic Regression with L1 regularization can be used for feature selection.

5. Principal Component Analysis (PCA):
   - PCA is a dimensionality reduction technique that transforms the original features into a new set of uncorrelated variables called principal components.
   - By selecting a subset of the principal components that explain most of the variance in the data, feature selection is implicitly performed.
   - PCA can be useful when dealing with high-dimensional data or when there is multicollinearity among features.

6. Forward/Backward Stepwise Selection:
   - Stepwise selection methods build a model iteratively by adding or removing features based on their impact on the model's performance.
   - Forward selection starts with an empty set of features and adds one feature at a time, selecting the one that improves the model the most.
   - Backward elimination starts with all features and iteratively removes the least significant feature until no further improvement is observed.

These are just some of the commonly used methods for feature selection in supervised learning classification tasks. The choice of method depends on the specific problem, dataset characteristics, and the type of model being used. It's often beneficial to experiment with different feature selection techniques and evaluate their impact on the model's performance.